### prompt 1

Gere em python um algoritmo que observe os seguintes requisitos:
1. Crie um algoritmo de Naive Bayes from scratch;
2. Não utilize bibliotecas como scikit-learn ou quaisquer outras que implementem tal algoritmo;
3. Utilize quando necessário bibliotecas de manipulação de dados como Pandas e Numpy;
4. Os atributos poderão ser tanto categóricos, como numéricos (inteiro e float);
5. Sem utilizar bibliotecas como scikit-learn, crie um algoritmo que retorne a matriz de confusão;
6. Gere funções em Python para calcular as seguintes métricas: acurácia, recall, precisão. Novamente, sem utilizar bibliotecas como scikit-learn.

### Resposta ao prompt 1

Para implementar o Naïve Bayes do zero, vamos considerar o cenário de classificação, onde o algoritmo utiliza o teorema de Bayes para calcular a probabilidade de uma amostra pertencer a uma determinada classe com base nos atributos fornecidos.

Aqui está o código em Python que implementa essa lógica:

In [1]:
import numpy as np
import pandas as pd

class NaiveBayes:
    def __init__(self):
        self.classes = None
        self.mean = None
        self.var = None
        self.priors = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        self.mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)

    def _gaussian_density(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            posterior = np.sum(np.log(self._gaussian_density(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        return np.array([self._predict(x) for x in X])

### prompt 2

Utilize o dataset Titanic para aplicar o algoritmo de Naive Bayes implementado.

In [2]:
df = pd.read_csv('titanic.csv')

# Pré-processamento dos dados
df = df[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex']]
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
df['Age'].fillna(df['Age'].mean(), inplace=True)

# Dividir em features (X) e target (y)
X = df.drop('Survived', axis=1).values
y = df['Survived'].values

# Dividir o dataset em treino e teste manualmente
def train_test_split_manual(X, y, test_size=0.2):
    n_samples = len(y)
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    test_set_size = int(n_samples * test_size)
    test_indices = indices[:test_set_size]
    train_indices = indices[test_set_size:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train, X_test, y_train, y_test = train_test_split_manual(X, y, test_size=0.2)

# Instanciar e treinar o Naive Bayes
nb = NaiveBayes()
nb.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = nb.predict(X_test)

# Calcular acurácia
accuracy = np.mean(y_pred == y_test)
print(f'Acurácia: {accuracy:.4f}')

### prompt 3

Adicione validação cruzada (k-fold cross-validation).

In [3]:
def cross_validation(modelo, X, y, k=5):
    fold_size = len(y) // k
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    
    scores = []
    
    for i in range(k):
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate((indices[:i * fold_size], indices[(i + 1) * fold_size:]))

        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        acc = np.mean(y_pred == y_test)
        scores.append(acc)
        print(f"Fold {i + 1} Acurácia: {acc:.4f}")

    avg_score = np.mean(scores)
    print(f"Média da acurácia após {k} folds: {avg_score:.4f}")
    return avg_score

# Executar k-fold cross-validation
cross_validation(nb, X, y, k=5)

### prompt 4

Faça sem scikit-learn

### Resposta ao prompt 4

O código já foi implementado sem scikit-learn. As métricas de avaliação podem ser calculadas manualmente, e aqui está uma função para gerar a matriz de confusão:

In [4]:
def confusion_matrix(y_true, y_pred):
    classes = np.unique(y_true)
    matrix = np.zeros((len(classes), len(classes)), dtype=int)
    
    for true, pred in zip(y_true, y_pred):
        matrix[true, pred] += 1
    return matrix

# Gerar matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(conf_matrix)

### Conclusão

O código implementa um algoritmo Naive Bayes do zero e aplica a validação cruzada, além de gerar a matriz de confusão e calcular as métricas de acurácia, recall e precisão.